# Preámbulo 

In [4]:
using LinearAlgebra
using Plots

In [5]:
# Primero definimos algunas funciones auxiliares que nos ayudarán a convertir de un número a una lista de bits y viceversa.
function original_integer(list)
    return parse(Int, join(list); base=2)
end

function base_2(integer; pad= nothing)
    if pad == nothing
        return reverse(digits(integer, base = 2))
    else
        return reverse(digits(integer, base = 2, pad = pad))
    end
end

base_2 (generic function with 1 method)

## Hola

In [6]:
A = base_2(5; pad=nothing)

3-element Vector{Int64}:
 1
 0
 1

In [7]:
original_integer([1,1,1,1,1])

31

In [8]:
function rotate(index,N)
    vec = base_2(index, pad=N)
    return original_integer([vec[end];vec[1:end-1]]), index
end

rotate (generic function with 1 method)

In [9]:
rotate(1,3)

(4, 1)

In [10]:
m = 3 # Número de Qubits
am = original_integer(fill(1,m))
A = zeros(am+1,am+1) #la primer posición es el cero

for i in 0:am # m es el número de qubits 
    A[ rotate(i,m)[1]+1, i+1 ] = 1
end
A

8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [11]:
# pad = 5, tendré 5 dígitos en la base binaria, por lo tanto 5 qubits, con 5 qubits se puede escribir hasta el número 31
typeof(rotate(1,3)[2])
am

7

In [12]:
N = 4
num = original_integer(fill(1,N))
V = zeros(N,num)
for i in 1:num
   V[:,i]=  base_2(i,pad=N)
end
V

4×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  1.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0
 1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0  1.0

In [13]:
num

15

In [14]:
# Definición de las matrices de Pauli
sigmax = [0 1.0; 1 0]
sigmay = [0 -1.0im; im 0]  # im representa la unidad imaginaria √(-1)
sigmaz = [1.0 0; 0 -1]
sigmas = Dict(1=>sigmax, 2=>sigmay, 3=>sigmaz)
J = 1
h = 1
function Sigma(indice, pos, totalparticulas)
    id = I(2) #[1.0 0; 0 1.0]
    mat = sigmas[indice]
    list = []
    for i in 0:totalparticulas-1
        if i == pos 
            append!(list,[mat])
        else 
            append!(list,[id])
        end
        #println("$list")
    end
    return kron(list...)
end

# Para la cadena abierta con n partículas tenemos usamos cerrada = false, para la cadena cerrada usamos cerrada = true
function Ising(N; cerrada = false) #Keyword Arguments
    H = 0
    totalparticulas = N
    if cerrada == false 
        M = (N - 1)
    elseif cerrada == true 
        M = N 
    end

    for i in 0:(M-1)
        H = H .+ J*(Sigma(3, mod(i,N),totalparticulas)*Sigma(3,mod((i+1),N),totalparticulas))
    end
    for j in 0:N-1
        H = H .+ h*Sigma(1,j,totalparticulas)
    end
    return H 
end

Ising (generic function with 1 method)

In [15]:
Ising(3, cerrada = true)

8×8 Matrix{Float64}:
 3.0   1.0   1.0   0.0   1.0   0.0   0.0  0.0
 1.0  -1.0   0.0   1.0   0.0   1.0   0.0  0.0
 1.0   0.0  -1.0   1.0   0.0   0.0   1.0  0.0
 0.0   1.0   1.0  -1.0   0.0   0.0   0.0  1.0
 1.0   0.0   0.0   0.0  -1.0   1.0   1.0  0.0
 0.0   1.0   0.0   0.0   1.0  -1.0   0.0  1.0
 0.0   0.0   1.0   0.0   1.0   0.0  -1.0  1.0
 0.0   0.0   0.0   1.0   0.0   1.0   1.0  3.0

In [16]:
vals, vecs = eigen(Ising(N, cerrada = true));
rounded_vals = round.(vals,digits=5)
final_vals = []
for (val, round_val) in zip(vals, rounded_vals)
    if norm(round_val - val) < 1e-3
        push!(final_vals, round_val)
    else
        push!(final_vals, val)
    end
end
final_vals_unique=unique(final_vals)

12-element Vector{Any}:
 -5.22625
 -4.82843
 -2.16478
 -2.0
 -0.82843
 -0.0
  0.0
  0.82843
  2.0
  2.16478
  4.82843
  5.22625

In [17]:
@show findall(vals .== final_vals_unique[3])


findall(vals .== final_vals_unique[3]) = Int64[]


Int64[]